# Custom Tools: Extending Agent Capabilities

In this notebook, we'll learn how to create custom tools using in-process MCP servers. Custom tools allow you to extend Claude's capabilities with your own specialized functionality.

## Setup

First, let's set up our environment:

In [5]:
# Setup for running async code in Jupyter
import nest_asyncio
nest_asyncio.apply()

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

print("✓ Notebook environment configured")

✓ Notebook environment configured


In [6]:
import os

# Verify API key
api_key = os.environ.get("ANTHROPIC_API_KEY")
if api_key:
    print(f"✓ API key found (length: {len(api_key)} characters)")
else:
    print("✗ API key not found. Please set ANTHROPIC_API_KEY environment variable.")

✓ API key found (length: 108 characters)


## Helper Function

Let's create a helper to print messages cleanly:

In [7]:
import json

def print_message(message):
    """Pretty print agent messages."""
    msg_type = type(message).__name__
    
    if msg_type == "SystemMessage":
        pass  # Skip system messages
    
    elif msg_type == "AssistantMessage":
        if hasattr(message, 'content'):
            for block in message.content:
                block_type = type(block).__name__
                if block_type == "TextBlock":
                    print(f"🤖 Assistant: {block.text}")
                elif block_type == "ToolUseBlock":
                    print(f"🔧 Tool: {block.name}")
                    if hasattr(block, 'input') and 'description' in block.input:
                        print(f"   → {block.input['description']}")
    
    elif msg_type == "UserMessage":
        if hasattr(message, 'content'):
            for block in message.content:
                block_type = type(block).__name__
                if block_type == "ToolResultBlock":
                    if block.is_error:
                        print(f"❌ Tool Error: {block.content}")
                    else:
                        content = str(block.content)
                        if len(content) > 300:
                            content = content[:300] + "..."
                        print(f"📤 Tool Result: {content}")
    
    elif msg_type == "ResultMessage":
        if hasattr(message, 'total_cost_usd') and hasattr(message, 'duration_ms'):
            print(f"\n💰 Cost: ${message.total_cost_usd:.4f} | ⏱️ Time: {message.duration_ms/1000:.1f}s")

## What Are Custom Tools?

Custom tools allow you to extend Claude's capabilities by:

- **Connecting to external APIs** (weather, databases, payment systems)
- **Adding domain-specific functionality** (calculations, data transformations)
- **Integrating with your services** (internal APIs, microservices)

### How It Works

1. **Define** tools using the `@tool` decorator
2. **Create** an MCP server with `create_sdk_mcp_server()`
3. **Pass** the server to `query()` via `mcp_servers` parameter
4. **Control** which tools are available via `allowed_tools`

### Tool Name Format

When exposed to Claude, tool names follow this pattern:
```
mcp__{server_name}__{tool_name}
```

Example: `get_weather` in server `my-tools` becomes `mcp__my-tools__get_weather`

## Example 1: Simple Weather Tool

Let's start with a simple tool that fetches weather data from a public API:

In [8]:
from claude_agent_sdk import tool, create_sdk_mcp_server
from typing import Any
import aiohttp

# Define a custom tool using the @tool decorator
@tool(
    "get_weather",
    "Get current temperature for a location using coordinates",
    {"latitude": float, "longitude": float}
)
async def get_weather(args: dict[str, Any]) -> dict[str, Any]:
    """Fetch weather data from Open-Meteo API."""
    try:
        async with aiohttp.ClientSession() as session:
            url = f"https://api.open-meteo.com/v1/forecast?latitude={args['latitude']}&longitude={args['longitude']}&current=temperature_2m&temperature_unit=fahrenheit"
            async with session.get(url) as response:
                data = await response.json()
        
        return {
            "content": [{
                "type": "text",
                "text": f"Temperature: {data['current']['temperature_2m']}°F"
            }]
        }
    except Exception as e:
        return {
            "content": [{
                "type": "text",
                "text": f"Error fetching weather: {str(e)}"
            }]
        }

# Create an MCP server with the custom tool
weather_server = create_sdk_mcp_server(
    name="weather-tools",
    version="1.0.0",
    tools=[get_weather]
)

print("✓ Weather tool created")

✓ Weather tool created


### Using the Weather Tool

Now let's use the weather tool with Claude. **Important**: MCP tools require streaming input mode (async generator), not a simple string.

In [10]:
from claude_agent_sdk import query, ClaudeAgentOptions

async def use_weather_tool():
    # Create an async generator for streaming input (required for MCP tools)
    async def message_generator():
        yield {
            "type": "user",
            "message": {
                "role": "user",
                "content": "What's the temperature in Singapore?"
            }
        }
    
    async for message in query(
        prompt=message_generator(),  # Use async generator
        options=ClaudeAgentOptions(
            mcp_servers={"weather-tools": weather_server},  # Pass as dict
            allowed_tools=[
                "mcp__weather-tools__get_weather"  # Tool name format
            ],
            permission_mode="bypassPermissions"
        )
    ):
        print_message(message)

await use_weather_tool()

🤖 Assistant: I'll get the current temperature in Singapore for you. Singapore is located at approximately latitude 1.29° N and longitude 103.85° E.
🔧 Tool: mcp__weather-tools__get_weather
📤 Tool Result: [{'type': 'text', 'text': 'Temperature: 87.5°F'}]
🤖 Assistant: The current temperature in Singapore is **87.5°F** (approximately 30.8°C).

💰 Cost: $0.0156 | ⏱️ Time: 5.7s


## Example 2: Calculator Tool with Multiple Functions

Let's create a more sophisticated tool with multiple mathematical operations:

In [11]:
import math

@tool(
    "calculate",
    "Perform mathematical calculations (basic arithmetic)",
    {"expression": str, "precision": int}
)
async def calculate(args: dict[str, Any]) -> dict[str, Any]:
    """Safely evaluate mathematical expressions."""
    try:
        # Use a restricted eval environment for safety
        safe_dict = {
            "__builtins__": {},
            "math": math,
            "abs": abs,
            "round": round,
            "min": min,
            "max": max,
            "sum": sum,
            "pow": pow
        }
        
        result = eval(args["expression"], safe_dict)
        precision = args.get("precision", 2)
        formatted = round(result, precision)
        
        return {
            "content": [{
                "type": "text",
                "text": f"{args['expression']} = {formatted}"
            }]
        }
    except Exception as e:
        return {
            "content": [{
                "type": "text",
                "text": f"Error: Invalid expression - {str(e)}"
            }]
        }

@tool(
    "compound_interest",
    "Calculate compound interest for an investment",
    {"principal": float, "rate": float, "time": float, "n": int}
)
async def compound_interest(args: dict[str, Any]) -> dict[str, Any]:
    """Calculate compound interest with detailed breakdown."""
    try:
        principal = args["principal"]
        rate = args["rate"]
        time = args["time"]
        n = args.get("n", 12)  # Default to monthly compounding
        
        amount = principal * (1 + rate / n) ** (n * time)
        interest = amount - principal
        
        result = f"""Investment Analysis:
Principal: ${principal:.2f}
Rate: {rate * 100:.2f}%
Time: {time} years
Compounding: {n} times per year

Final Amount: ${amount:.2f}
Interest Earned: ${interest:.2f}
Return: {(interest / principal) * 100:.2f}%"""
        
        return {
            "content": [{"type": "text", "text": result}]
        }
    except Exception as e:
        return {
            "content": [{"type": "text", "text": f"Error: {str(e)}"}]
        }

# Create calculator server with both tools
calculator_server = create_sdk_mcp_server(
    name="calculator",
    version="1.0.0",
    tools=[calculate, compound_interest]
)

print("✓ Calculator tools created")

✓ Calculator tools created


### Using Calculator Tools

Let's see both calculator tools in action:

In [12]:
async def use_calculator_tools():
    async def message_generator():
        yield {
            "type": "user",
            "message": {
                "role": "user",
                "content": "Calculate sqrt(144) + 25^2, and tell me how much $10,000 grows to at 5% annual interest over 10 years with monthly compounding"
            }
        }
    
    async for message in query(
        prompt=message_generator(),
        options=ClaudeAgentOptions(
            mcp_servers={"calculator": calculator_server},
            allowed_tools=[
                "mcp__calculator__calculate",
                "mcp__calculator__compound_interest"
            ],
            permission_mode="bypassPermissions"
        )
    ):
        print_message(message)

await use_calculator_tools()

🤖 Assistant: I'll calculate both of those for you.
🔧 Tool: mcp__calculator__calculate
🔧 Tool: mcp__calculator__compound_interest
📤 Tool Result: [{'type': 'text', 'text': "Error: Invalid expression - name 'sqrt' is not defined"}]
📤 Tool Result: [{'type': 'text', 'text': 'Investment Analysis:\nPrincipal: $10000.00\nRate: 500.00%\nTime: 10 years\nCompounding: 12 times per year\n\nFinal Amount: $14194530740392836464640.00\nInterest Earned: $14194530740392836464640.00\nReturn: 141945307403928371200.00%'}]
🤖 Assistant: Let me try the first calculation with a different approach:
🔧 Tool: mcp__calculator__calculate
🔧 Tool: mcp__calculator__compound_interest
📤 Tool Result: [{'type': 'text', 'text': '144**(1/2) + 25**2 = 637.0'}]
📤 Tool Result: [{'type': 'text', 'text': 'Investment Analysis:\nPrincipal: $10000.00\nRate: 5.00%\nTime: 10 years\nCompounding: 12 times per year\n\nFinal Amount: $16470.09\nInterest Earned: $6470.09\nReturn: 64.70%'}]
🤖 Assistant: Perfect! Here are your answers:

1. **s